## Dibels benchmark analysis for elementary school students  

The Dibels 8 assessment test helps teachers understand how well their students are acquiring literacy skills. Analysis found that in pre-pandemic about 56% of all test takers met grade level reading expectations. This fell by seven percentage points to 49% in 2020-2021. Among them Latino and Pacific Islander students suffered the most, dropping about eight percentage points. 

In [1]:
import pandas as pd
import altair as alt

In [2]:
dibels_raw = pd.read_csv("input/dibels.csv", thousands=',')

In [3]:
dibels_raw.columns = [ c.replace("\r","") for c in dibels_raw.columns]

Trim to middle of year 2019-2020 and 2020-2021

In [4]:
dibels_combined= dibels_raw[(dibels_raw.SchoolYear != "2018-19")&(dibels_raw.DIBELSTestAdministration == '2 MOY')].copy()

Combine above and met benchmark, and combine all below benchmark

In [5]:
dibels_combined["met_or_above"] =dibels_combined[['ABOVEBENCHMARK', 'BENCHMARK']].sum(axis=1)

In [6]:
dibels_combined["below_benchmark"] =dibels_combined[['BELOWBENCHMARK', 'WELLBELOWBENCHMARK']].sum(axis=1)

Calculate percentage

In [7]:
dibels_combined["met_or_above_pct"] = dibels_combined["met_or_above"]/dibels_combined['Total Tested']*100

In [8]:
dibels_combined["below_benchmark_pct"] =dibels_combined["below_benchmark"]/dibels_combined['Total Tested']*100

In [9]:
dibels_combined["met_or_above_change"] = dibels_combined.sort_values("SchoolYear").groupby(["Student Group"])["met_or_above_pct"].diff()

In [10]:
dibels_combined["below_benchmark_change"] = dibels_combined.sort_values("SchoolYear").groupby(["Student Group"])["below_benchmark_pct"].diff()

In [11]:
dibels_combined.SchoolYear.value_counts()

2019-20    18
2020-21    18
Name: SchoolYear, dtype: int64

Pivot for chart

In [12]:
dibels_chart = dibels_combined[["SchoolYear","Student Group","met_or_above_pct"]].pivot(index='SchoolYear', columns='Student Group').reset_index().droplevel(0, axis=1)

In [13]:
dibels_chart.columns

Index(['', 'African American', 'All', 'American Indian /Alaska Native',
       'Asian', 'English Learner', 'Filipino', 'Grade 1', 'Grade 2', 'Grade 3',
       'Grade 4', 'Grade 5', 'Grade K', 'Grade TK', 'Hispanic',
       'Other (Ethnicity)', 'Pacific Islander', 'Two or More (Ethnicity)',
       'White'],
      dtype='object', name='Student Group')

In [14]:
dibels_chart

Student Group,,African American,All,American Indian /Alaska Native,Asian,English Learner,Filipino,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade K,Grade TK,Hispanic,Other (Ethnicity),Pacific Islander,Two or More (Ethnicity),White
0,2019-20,48.297117,55.491021,53.939394,78.243021,31.587858,79.344859,62.844200,64.640022,57.802040,44.420487,52.419085,54.853748,25.028902,51.385985,49.790210,62.264151,72.931580,72.459255
1,2020-21,42.486646,48.827828,46.979866,78.684323,26.089621,76.760351,45.838748,53.626691,53.205859,56.130128,50.105232,36.427994,15.633546,43.306432,44.029851,53.910615,70.427692,70.233166


In [15]:
dibels_chart_trim = dibels_chart[['','African American','English Learner', 'Hispanic',
       'White', 'Asian']]

In [16]:
dibels_chart_trim.to_csv("output/dibels_chart.csv",index=False)

In [17]:
dibels_chart_trim

Student Group,,African American,English Learner,Hispanic,White,Asian
0,2019-20,48.297117,31.587858,51.385985,72.459255,78.243021
1,2020-21,42.486646,26.089621,43.306432,70.233166,78.684323


In [18]:
alt.Chart(dibels_combined, title="2019/ 2020 comparison").mark_point().encode(
    alt.X(
        'met_or_above_pct:Q',
        title="Met or above %",
        scale=alt.Scale(zero=False),
        axis=alt.Axis(grid=False)
    ),
    alt.Y(
        'Student Group:N',
        title="",
        sort='x',
        axis=alt.Axis(grid=True)
    ),
    color=alt.Color('SchoolYear:N', legend=alt.Legend(title="Year")),
).properties(
    height=alt.Step(20)
).configure_view(stroke="transparent")

alt.Chart(...)